In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/train1.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

train_dataset = dataset.select(range(5000))
test_dataset = dataset.select(range(5000, len(dataset), 1))


In [ ]:
!pip install evaluate
!pip install datasets
!pip install transformers
!pip install accelerate -U


In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
# dataset = load_dataset("csv", data_files="/content/train1.csv", split="train")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


id2label = {0: "NO NEED INTERNET", 1: "NEED INTERNET"}
label2id = {"NO NEED INTERNET": 0, "NEED INTERNET": 1}


tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-uncased', num_labels=2, id2label=id2label, label2id=label2id)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

model.resize_token_embeddings(len(tokenizer))

trainer = Trainer(
    model,
    train_dataset=tokenized_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args = TrainingArguments(
        # learning_rate=2e-5,
        # per_device_train_batch_size=16,
        # per_device_eval_batch_size=16,
        num_train_epochs=4,
        # weight_decay=0.01,
        # evaluation_strategy="epoch",
        save_strategy="epoch",
        # load_best_model_at_end=True,
        # push_to_hub=True,
        per_device_train_batch_size = 1,
        # save_strategy = 'steps',
        # save_steps = 1000,
        output_dir = 'drive/MyDrive/gpt-oasst'
    ),
)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.297600
1000,0.329700
1500,0.313600
2000,0.180700
2500,0.169700
3000,0.238000
3500,0.148900
4000,0.219200
4500,0.168800
5000,0.192900


Checkpoint destination directory drive/MyDrive/gpt-oasst/checkpoint-5000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=20000, training_loss=0.10750168617665767, metrics={'train_runtime': 879.7425, 'train_samples_per_second': 22.734, 'train_steps_per_second': 22.734, 'total_flos': 62285549907120.0, 'train_loss': 0.10750168617665767, 'epoch': 4.0})

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained( "/content/drive/MyDrive/gpt-oasst/checkpoint-15000")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt-oasst/checkpoint-15000")

true_answers = 0
tp = 0
fp = 0
tn = 0
fn = 0
for question in test_dataset:
  text = question['text']
  inputs = tokenizer(text, return_tensors= "pt")
  with torch.no_grad():
    logits = model(**inputs).logits

  predicted_class_id = logits.argmax().item()
  if predicted_class_id == question['label'] and predicted_class_id == 1:
    tp += 1
  elif predicted_class_id == question['label'] and predicted_class_id == 0:
    tn += 1
  elif predicted_class_id != question['label'] and predicted_class_id == 1:
    fp += 1
  else:
    fn += 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

# accuracy = true_answers / len(test_dataset)

# accuracy
f1_score

0.976027397260274

In [ ]:
!zip -r checkpoint-79000.zip /content/drive/MyDrive/gpt-oasst/checkpoint-20000

  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/ (stored 0%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/tokenizer_config.json (deflated 75%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/trainer_state.json (deflated 80%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/config.json (deflated 48%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/tokenizer.json (deflated 71%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/rng_state.pth (deflated 25%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/training_args.bin (deflated 51%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/vocab.txt (deflated 53%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/scheduler.pt (deflated 56%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/model.safetensors (deflated 8%)
  adding: content/drive/MyDrive/gpt-oasst/checkpoint-20000/optimizer.pt (deflated 36%)
  adding: content/drive/MyDrive/gpt